# Getting started

In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

wdir = os.path.join(os.getcwd(), "3_0", "McMurdo")
#infile = os.path.join(wdir, "lawrence09.v30.txt")
#ipmag.download_magic(infile, overwrite=True)


/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df


# Several ways of creating a contribution

In [2]:
reload(nb)

# test out various ways of creating a contribution

#class Contribution(object):
#    """                                                                                                                   
#    A Contribution is a collection of MagicDataFrames,                                                                    
#    each of which corresponds to one MagIC table.                                                                         
#    The Contribution object also has methods for                                                                          
#    manipulating one or more tables in the contribution --                                                                
#    for example, renaming a site.                                                                                         
#    """
#    def __init__(self, directory, read_tables='all',
#                 custom_filenames=None, single_file=None):



# make contribution reading in all default filenames from working directory
con = nb.Contribution(wdir)
print 'tables created:', con.tables.keys()
print '-'

# make contribution with some custom filenames
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt'})
print 'tables created:', con.tables.keys()
print '-'

# make contribution with custom filenames, and only read in the specimen table to start
con = Contribution(wdir, read_tables=['specimens'], custom_filenames={'sites': 'custom_sites.txt',
                                                                      'specimens': 'custom_specimens.txt'})
print 'tables created:', con.tables.keys()
print '-'

# make contribution with a single, mystery file (can be any datatype)
con = nb.Contribution(wdir, single_file='sites.txt')
print 'tables created:', con.tables.keys()
print '-'


-W- No such file: /Users/nebula/Python/PmagPy/3_0/McMurdo/locations.txt
tables created: ['measurements', 'ages', 'sites', 'samples', 'criteria', 'images', 'contribution', 'specimens']
-
-W- No such file: /Users/nebula/Python/PmagPy/3_0/McMurdo/locations.txt
tables created: ['measurements', 'ages', 'sites', 'samples', 'criteria', 'images', 'contribution', 'specimens']
-
tables created: ['specimens']
-
tables created: ['sites']
-


In [3]:
# make McMurdo contribution, starting with specimens table

reload(nb)

con = nb.Contribution(wdir, read_tables=['specimens'], custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                                                         'sites': 'custom_sites.txt'})

print con.filenames
print con.tables.keys()

{'measurements': 'measurements.txt', 'ages': 'ages.txt', 'sites': 'custom_sites.txt', 'locations': 'locations.txt', 'samples': 'custom_samples.txt', 'criteria': 'criteria.txt', 'images': 'images.txt', 'contribution': 'contribution.txt', 'specimens': 'custom_specimens.txt'}
['specimens']


In [4]:
# then, add another table to the contribution
# here, we are providing data type but no filename
# this works because we already gave the custom sample filename when we created the contribution
# so the contribution already knows where to look (con.filenames)
con.add_magic_table('samples')
print con.tables.keys()

['specimens', 'samples']


In [5]:
# add another table to the same contribution
# this time, provide a filename but no data type

con.add_magic_table(dtype="unknown", fname="criteria.txt")
# criteria table now included
print con.tables.keys()

['specimens', 'samples', 'criteria']


# Functionality with a contribution

In [6]:
# create full McMurdo contribution

reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})


-W- No such file: /Users/nebula/Python/PmagPy/3_0/McMurdo/locations.txt


In [7]:
con.tables['locations'].df[['sites']]

KeyError: 'locations'

## Rename an item

In [ ]:
reload(nb)
# rename one of the Contribution's sites
con.rename_item('sites', 'mc01', 'extra_special_site')
con.tables['sites'].df.ix[['extra_special_site']]
# all rows previously named 'mc01' are now named 'extra_special_site'

In [ ]:
# additionally, 'mc01' has been replaced in the location table under site_names
#con.tables['locations'].df.ix[["Osler Volcanics, Nipigon Strait, Lower Reversed"]][['site_names']]
con.tables['locations'].df[['sites']]#, 'sites_list']]

## Propagate data from one table into another

In [ ]:

# normally, each table only has one relationship up (i.e., a measurement table will have specimen name, but not sample name)
# sometimes, you need to access location_name at the site level (for example)
# this function propagates names down through any available tables
# the code snippet below won't work if the Contribution can't access the sample and site files!


reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

con.propagate_name_down('location', 'specimens')

# specimens table now has sample, site, and location_names
con.tables['specimens'].df[['specimen', 'sample', 'site', 'location']].head()


In [ ]:
# this function propagates values from arbitrary columns down
# i.e., get sample-level azimuth into the measurements table
# note: this will NOT work with names (specimen_name, sample_name, etc.).  
# for those relationships, use the above function: propagate_name_down

reload(nb)
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

meas_container = con.tables['measurements']
meas_df = meas_container.df

meas_df = con.propagate_cols_down(['azimuth', 'dip', 'fake_col'], 'measurements', 'samples')
meas_df.head()[['azimuth', 'dip']]

## Writing out a MagIC file

In [ ]:
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})
site_container = con.tables['sites']
site_container.write_magic_file(custom_name='fancy_sites.txt', dir_path='./3_0/McMurdo')

site_container.df

In [ ]:
site_container.df.index.name

## Mucking around with measurements

In [ ]:
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})

meas_container = con.tables['measurements']
meas_data = meas_container.df

meas_data['treatment'] = meas_data['treat_ac_field'].where(cond=meas_data['treat_ac_field'] != "0", other=meas_data['treat_temp'])
meas_data[['treatment', 'treat_ac_field', 'treat_temp']]

meas_data['treat_ac_field'].ix[0] = None
meas_data['treat_ac_field'] = meas_data['treat_ac_field'].astype(float)

meas_data[['treatment', 'treat_ac_field', 'treat_temp']].head()


## Dealing with criteria

In [ ]:
#  grab a copy of the criteria and sites table to play with
criteria = con.tables['criteria'].df.copy()
sites = con.tables['sites'].df.copy()
locations = con.tables['locations'].df.copy()
specimens = con.tables['specimens'].df.copy()
samples = con.tables['samples'].df.copy()
sites.head()

In [ ]:
# get all criteria for sites

# only criteria with 'site' in table_column_name
cond = criteria.index.str.contains('site')
site_crit = criteria[cond].copy()
# remove table name from index
site_crit.index = site_crit.index.str.replace('sites.', '')
site_crit.index.name = 'column_name'


cols = site_crit.index
list(cols)
cols = list(cols)
cols.append('criteria_names')
site_crit

In [ ]:
sites.head()[cols]

In [ ]:
# create string --> operator conversion    
import operator
ops = {"<": operator.lt, ">": operator.gt, "==": operator.eq, "<=": operator.le, ">=": operator.gt}

# function for applying criteria
def apply_crit(series, crit_series, criteria_type):
    """
    Apply 1 criterion (i.e., 1 row of the criteria table) to another table.
    Return series with boolean values for whether the row passes.
    """
    col_name = crit_series.name
    #print col_name
    # if there's no value, pass == True??  or == False?
    if not series[col_name]:
        return "{} not in row".format(col_name)
    # if we're missing criteria names, then what??
    elif not series['criteria_names']:
        return "no value in criteria_names"
    elif criteria_type not in series['criteria_names']:
        return "{} not in criteria_names for this row".format(criteria_type)
    crit_name = crit_series['criterion']
    crit_value = float(crit_series['criterion_value'])
    op_str = crit_series['criterion_operation']
    op = ops[op_str]
    value = float(series[col_name])
    #print value, op_str, crit_value
    result = op(value, crit_value)
    #print op
    #print result
    return result





In [ ]:
# apply a criterion to a table
crit_name = 'dir_alpha95'
pass_name = 'pass_' + crit_name
crit1 = site_crit.ix[crit_name]
sites[pass_name] = sites.apply(apply_crit, axis=1, args=(crit1, 'DE-SITE'))
cond = sites[pass_name] == True
sites[cond][[crit_name, pass_name]].head()

In [ ]:
for crit_name in site_crit.index[1:]:
    crit_series = site_crit.ix[crit_name]
    # if there are multiple records for a single crit_name, ignore that one
    if not isinstance(crit_series, pd.Series):
        continue
    sites['pass_' + crit_name] = sites.apply(apply_crit, axis=1, args=(crit_series, "DE-SITE"))

    
cond = sites.columns.str.contains('pass')
sites.head()
sites[sites.columns[cond]].head()

In [ ]:
# create full McMurdo contribution

reload(nb)

con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                             'sites': 'custom_sites.txt'})


criteria = con.tables['criteria'].df.copy()


#criteria
#print 'criteria_names' in specimens.columns
#print 'criteria_names' in samples.columns
#print 'criteria_names' in sites.columns
#print 'criteria_names' in locations.columns
#sites[['criteria_names']]


def apply_crit(series, crit_series):#, criteria_type):
    """
    Apply 1 criterion (i.e., 1 row of the criteria table) to another table.
    Return series with boolean values for whether the row passes.
    """
    col_name = crit_series.name
    # if there's no value, pass == True
    if not series[col_name]:
        return True
        #return "{} not in row".format(col_name)
    # if there is a value, test that it is within correct limits
    crit_name = crit_series['criterion']
    crit_value = float(crit_series['criterion_value'])
    op_str = crit_series['criterion_operation']
    op = ops[op_str]
    value = float(series[col_name])
    result = op(value, crit_value)
    return result



def add_criteria_named(category_name, dtype):
    df = con.tables[dtype].df
    criteria_subset = criteria[criteria['criterion'] == category_name]
    criteria_subset.index = criteria_subset.index.str.replace(dtype + '.', '')
    pass_col_names = []
    for crit_name, crit_row in criteria_subset.iterrows():
        #print 'crit_name', crit_name
        col_name = category_name + "_" + crit_name + "_pass"
        #print 'col_name', col_name
        pass_col_names.append(col_name)
        df[col_name] = df.apply(apply_crit, args=(crit_row,), axis=1)
    return pass_col_names
    

#DE_SPEC = criteria[criteria['criterion'] == 'DE-SPEC']
#DE_SPEC.index = DE_SPEC.index.str.replace('specimens.', '')
#pass_col_names = []
#for crit_name, crit_row in DE_SPEC.iterrows():
#    #print 'crit_name', crit_name
#    col_name = 'DE-SPEC_' + crit_name + "_pass"
#    #print 'col_name', col_name
#    pass_col_names.append(col_name)
#    specimens[col_name] = specimens.apply(apply_crit, args=(crit_row,), axis=1)
    
    
#specimens_container = con.tables['specimens']
dtype = 'specimens'
criteria_name = 'IE-SPEC'
pass_col = criteria_name + "_pass"
pass_col_names = add_criteria_named(criteria_name, dtype)
df = con.tables[dtype].df

#df.ix[:, 13:]
col_names = df.columns[df.columns.str.contains(criteria_name)]
df[df[col_names].all(1)].head()#[col_names]

#cond1 = df[pass_col_names[0]]
#cond2 = df[pass_col_names[1]]
#cond3 = df[pass_col_names[2]]

#cond = cond1 & cond2 & cond3
#df[pass_col] = cond

## all specimens that pass all DE-SPEC criteria
#df[df[pass_col]].index



In [ ]:
reload(nb)
import pmagpy.pmag
reload(pmagpy.pmag)
con = nb.Contribution(wdir, custom_filenames={'specimens': 'custom_specimens.txt', 'samples': 'custom_samples.txt',
                                                                         'sites': 'custom_sites.txt'})



con.tables['ages'].df.head()

# Cleanup

In [ ]:
!rm sites.txt samples.txt specimens.txt measurements.txt ages.txt contribution.txt images.txt criteria.txt locations.txt
!rm *.png
!rm *.jpg